In [1]:
import pandas as pd
import numpy as np

In [13]:
df=pd.read_csv('Online_Sales.csv')
df.head()

,CustomerID,Transaction_ID,Transaction_Date,Product_SKU,Product_Description,Product_Category,Quantity,Avg_Price,Delivery_Charges,Coupon_Status
0,17850,16679,20190101,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1,153.71,6.5,Used
1,17850,16680,20190101,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1,153.71,6.5,Used
2,17850,16681,20190101,GGOEGFKQ020399,Google Laptop and Cell Phone Stickers,Office,1,2.05,6.5,Used
3,17850,16682,20190101,GGOEGAAB010516,Google Men's 100% Cotton Short Sleeve Hero Tee...,Apparel,5,17.53,6.5,Not Used
4,17850,16682,20190101,GGOEGBJL013999,Google Canvas Tote Natural/Navy,Bags,1,16.50,6.5,Used


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52924 entries, 0 to 52923
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CustomerID           52924 non-null  int64  
 1   Transaction_ID       52924 non-null  int64  
 2   Transaction_Date     52924 non-null  int64  
 3   Product_SKU          52924 non-null  object 
 4   Product_Description  52924 non-null  object 
 5   Product_Category     52924 non-null  object 
 6   Quantity             52924 non-null  int64  
 7   Avg_Price            52924 non-null  float64
 8   Delivery_Charges     52924 non-null  float64
 9   Coupon_Status        52924 non-null  object 
dtypes: float64(2), int64(4), object(4)
memory usage: 4.0+ MB


In [15]:
df['Transaction_Date']=pd.to_datetime(df['Transaction_Date'].astype(str),format='%Y%m%d')


The dataframe df is on product level so we will need to convert this df into transaction level df.

# 1. Customer Acquisition Cost (CAC)

In [48]:
spend=pd.read_csv('Marketing_Spend.csv')
spend['Date']=pd.to_datetime(spend['Date'],format='%m/%d/%Y')

In [49]:
tax=pd.read_excel('Tax_amount.xlsx')
disc=pd.read_csv('Discount_Coupon.csv')
df['Month']=df.Transaction_Date.dt.strftime('%b')

df1=df.merge(tax, how='inner',on='Product_Category').merge(disc, how='left',left_on=['Month','Product_Category'],right_on=['Month','Product_Category '])
def fun(row):
    return (row['Avg_Price'] * row['Quantity']) * (1 - (row['Discount_pct'] / 100)) * (1 + row['GST'])

# Amt is amount for each product purchased for each transaction_id 
# {The delivery charges are not yet included to get invoice or transaction amt for each transaction_id}
df1['Amt'] = df1.apply(fun, axis=1)

# Getting transaction level dataframe
trans_level_df=df1.groupby(['CustomerID','Transaction_ID','Transaction_Date']).agg(  products_purchased=('Product_SKU','unique'),       \
                products_categories=('Product_Category','unique'), Product_Total=('Amt','sum'), del_charge=('Delivery_Charges','mean')).reset_index()

trans_level_df['Trans_Total']=trans_level_df['Product_Total']+ trans_level_df['del_charge']
trans_level_df.head()

,CustomerID,Transaction_ID,Transaction_Date,products_purchased,products_categories,Product_Total,del_charge,Trans_Total
0,12346,38223,2019-09-15,"[GGOEAAAJ080816, GGOEGOAR013099]","[Apparel, Office]",24.98174,75.00,99.98174
1,12347,23979,2019-03-24,"[GGOEGOAQ020099, GGOEGOCT019199]",[Office],9.99460,14.41,24.40460
2,12347,23980,2019-03-24,"[GGOENEBJ079499, GGOENEBQ079199]",[Nest-USA],418.88000,6.50,425.38000
3,12347,23981,2019-03-24,"[GGOEGBFC018799, GGOEGFKA022299, GGOEGFYQ01659...","[Bags, Office, Drinkware, Headgear]",82.32644,49.73,132.05644
4,12347,23984,2019-03-24,"[GGOEGAEL031113, GGOEGAEL031114, GGOEGAEL03111...",[Apparel],7167.88758,6.50,7174.38758


In [50]:
cust=pd.read_excel('CustomersData.xlsx')
cac_calc=trans_level_df.merge(cust,on='CustomerID',how='inner')
cac_calc.head()

,CustomerID,Transaction_ID,Transaction_Date,products_purchased,products_categories,Product_Total,del_charge,Trans_Total,Gender,Location,Tenure_Months
0,12346,38223,2019-09-15,"[GGOEAAAJ080816, GGOEGOAR013099]","[Apparel, Office]",24.98174,75.00,99.98174,F,New York,31
1,12347,23979,2019-03-24,"[GGOEGOAQ020099, GGOEGOCT019199]",[Office],9.99460,14.41,24.40460,M,New York,20
2,12347,23980,2019-03-24,"[GGOENEBJ079499, GGOENEBQ079199]",[Nest-USA],418.88000,6.50,425.38000,M,New York,20
3,12347,23981,2019-03-24,"[GGOEGBFC018799, GGOEGFKA022299, GGOEGFYQ01659...","[Bags, Office, Drinkware, Headgear]",82.32644,49.73,132.05644,M,New York,20
4,12347,23984,2019-03-24,"[GGOEGAEL031113, GGOEGAEL031114, GGOEGAEL03111...",[Apparel],7167.88758,6.50,7174.38758,M,New York,20


In [90]:
cac_calc

,CustomerID,Transaction_ID,Transaction_Date,products_purchased,products_categories,Product_Total,del_charge,Trans_Total,Gender,Location,Tenure_Months
0,12346,38223,2019-09-15,"[GGOEAAAJ080816, GGOEGOAR013099]","[Apparel, Office]",24.98174,75.00,99.98174,F,New York,31
1,12347,23979,2019-03-24,"[GGOEGOAQ020099, GGOEGOCT019199]",[Office],9.99460,14.41,24.40460,M,New York,20
2,12347,23980,2019-03-24,"[GGOENEBJ079499, GGOENEBQ079199]",[Nest-USA],418.88000,6.50,425.38000,M,New York,20
3,12347,23981,2019-03-24,"[GGOEGBFC018799, GGOEGFKA022299, GGOEGFYQ01659...","[Bags, Office, Drinkware, Headgear]",82.32644,49.73,132.05644,M,New York,20
4,12347,23984,2019-03-24,"[GGOEGAEL031113, GGOEGAEL031114, GGOEGAEL03111...",[Apparel],7167.88758,6.50,7174.38758,M,New York,20
...,...,...,...,...,...,...,...,...,...,...,...
26626,18283,40286,2019-10-10,[GGOENEBQ086799],[Nest],93.55500,6.00,99.55500,F,New Jersey,36
26627,18283,40288,2019-10-10,"[GGOEGDHC018299, GGOEGFKQ020799, GGOEGOBG023599]","[Drinkware, Office]",36.38232,6.00,42.38232,F,New Jersey,36
26628,18283,40289,2019-10-10,"[GGOEGAYR068226, GGOEGHPB071610]",[Apparel],25.46676,6.50,31.96676,F,New Jersey,36
26629,18283,40290,2019-10-10,"[GGOEAAEJ030916, GGOEGAEB027814]",[Apparel],52.01676,6.00,58.01676,F,New Jersey,36


In [84]:
days=cac_calc.Transaction_Date.sort_index().unique()
new_cust={}
male_cust={}
female_cust={}
ny_cust={}
cal_cust={}
chic_cust={}
nj_cust={}
dc_cust={}
for day in days:
    if day== days.min():
        new_cust[day]=cac_calc[cac_calc.Transaction_Date==day].CustomerID.nunique()
        # Gender wise new cust
        male_cust[day]=cac_calc[(cac_calc.Transaction_Date==day) & (cac_calc.Gender=='M')].CustomerID.nunique()
        female_cust[day]= new_cust[day]-male_cust[day]
        # Location wise new cust
        ny_cust[day]=cac_calc[(cac_calc.Transaction_Date==day) & (cac_calc.Location=='New York')].CustomerID.nunique()
        cal_cust[day]=cac_calc[(cac_calc.Transaction_Date==day) & (cac_calc.Location=='California')].CustomerID.nunique()
        chic_cust[day]=cac_calc[(cac_calc.Transaction_Date==day) & (cac_calc.Location=='Chicago')].CustomerID.nunique()
        nj_cust[day]=cac_calc[(cac_calc.Transaction_Date==day) & (cac_calc.Location=='New Jersey')].CustomerID.nunique()
        dc_cust[day]= new_cust[day] - (nj_cust[day]+chic_cust[day]+cal_cust[day]+ny_cust[day])

    else:
        cust_ids=set(cac_calc[cac_calc.Transaction_Date == day].CustomerID)-set(cac_calc[cac_calc.Transaction_Date < day].CustomerID)
        new_cust[day]=len(cust_ids)

        # gender wise
        male_cust[day]= cac_calc[(cac_calc['CustomerID'].isin(cust_ids)) & (cac_calc.Gender=='M')].CustomerID.nunique()
        female_cust[day]=new_cust[day]-male_cust[day]
        # location wise
        ny_cust[day]=cac_calc[(cac_calc['CustomerID'].isin(cust_ids)) & (cac_calc.Location=='New York')].CustomerID.nunique()
        cal_cust[day]=cac_calc[(cac_calc['CustomerID'].isin(cust_ids)) & (cac_calc.Location=='California')].CustomerID.nunique()
        chic_cust[day]=cac_calc[(cac_calc['CustomerID'].isin(cust_ids)) & (cac_calc.Location=='Chicago')].CustomerID.nunique()
        nj_cust[day]=cac_calc[(cac_calc['CustomerID'].isin(cust_ids)) & (cac_calc.Location=='New Jersey')].CustomerID.nunique()
        dc_cust[day]= new_cust[day] - (nj_cust[day]+chic_cust[day]+cal_cust[day]+ny_cust[day])


            

In [91]:
cac=pd.DataFrame({
    'New_cust_gained': new_cust.values(),
    'Male_count':male_cust.values(),
    'Female_count': female_cust.values(),
    'NY_count': ny_cust.values(),
    'Cal_count': cal_cust.values(),
    'Chic_count': chic_cust.values(),
    'NJ_count': nj_cust.values(),
    'DC_count': dc_cust.values()   

}, index=new_cust.keys()
).sort_index()
cac

,New_cust_gained,Male_count,Female_count,NY_count,Cal_count,Chic_count,NJ_count,DC_count
2019-01-01,7,5,2,1,4,2,0,0
2019-01-02,9,2,7,1,3,2,2,1
2019-01-03,12,4,8,0,2,8,1,1
2019-01-04,4,0,4,0,4,0,0,0
2019-01-05,8,4,4,1,4,1,1,1
...,...,...,...,...,...,...,...,...
2019-12-27,2,1,1,0,0,1,1,0
2019-12-28,1,1,0,0,0,1,0,0
2019-12-29,0,0,0,0,0,0,0,0
2019-12-30,2,0,2,1,0,0,1,0


This is a distribution of daily customer gained based on gender and location

In [93]:
spend['Total_Spend']=spend.iloc[:,1]+ spend.iloc[:,2]
cac= spend.merge(cac, how='inner', left_on='Date',right_index=True)
cac['Customer Acquisition Cost (CAC)']=(cac['Total_Spend']/cac['New_cust_gained']).round(2)
cac

,Date,Offline_Spend,Online_Spend,Total_Spend,New_cust_gained,Male_count,Female_count,NY_count,Cal_count,Chic_count,NJ_count,DC_count,Customer Acquisition Cost (CAC)
0,2019-01-01,4500,2424.50,6924.50,7,5,2,1,4,2,0,0,989.21
1,2019-01-02,4500,3480.36,7980.36,9,2,7,1,3,2,2,1,886.71
2,2019-01-03,4500,1576.38,6076.38,12,4,8,0,2,8,1,1,506.36
3,2019-01-04,4500,2928.55,7428.55,4,0,4,0,4,0,0,0,1857.14
4,2019-01-05,4500,4055.30,8555.30,8,4,4,1,4,1,1,1,1069.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2019-12-27,4000,3396.87,7396.87,2,1,1,0,0,1,1,0,3698.44
361,2019-12-28,4000,3246.84,7246.84,1,1,0,0,0,1,0,0,7246.84
362,2019-12-29,4000,2546.58,6546.58,0,0,0,0,0,0,0,0,inf
363,2019-12-30,4000,674.31,4674.31,2,0,2,1,0,0,1,0,2337.15


Here we have daily amount we spent on acquiring one customer.


# 2. Marketing Efficiency Ratio (MER) or Revenue to marketing spend ratio 

In [291]:
# Revenue each day
rev_pd=trans_level_df.groupby('Transaction_Date').agg(Revenue=('Trans_Total','sum'))
mer=spend.merge(rev_pd, how='inner',left_on='Date',right_index=True)
mer['Marketing Eff Ratio']=(mer['Revenue']/mer['Total_Spend']).round(3)
mer

,Date,Offline_Spend,Online_Spend,Total_Spend,Revenue,Marketing Eff Ratio
0,2019-01-01,4500,2424.50,6924.50,7768.31708,1.122
1,2019-01-02,4500,3480.36,7980.36,13748.03838,1.723
2,2019-01-03,4500,1576.38,6076.38,25326.11859,4.168
3,2019-01-04,4500,2928.55,7428.55,17174.12365,2.312
4,2019-01-05,4500,4055.30,8555.30,18608.13618,2.175
...,...,...,...,...,...,...
360,2019-12-27,4000,3396.87,7396.87,10939.79407,1.479
361,2019-12-28,4000,3246.84,7246.84,8730.70022,1.205
362,2019-12-29,4000,2546.58,6546.58,9664.96779,1.476
363,2019-12-30,4000,674.31,4674.31,6442.21505,1.378


# 3) Customer Lifetime Value (CLV) & 4) Average Order Value (AOV)

In [11]:
# Creating Basket ID
df2=trans_level_df.groupby(['CustomerID', 'Transaction_Date']).agg(total_spend_each_day= ('Trans_Total','sum')).reset_index()
df2['Basket_ID']=df2.index +1000
df2

'''Here Basket_ID is calculated by considering full day as a shopping session. If the time info of the purchase were also known to us we 
could have used different approach and such an approach would also have helped us to comment on the peak hours revenue etc.  '''

,CustomerID,Transaction_Date,total_spend_each_day,Basket_ID
0,12346,2019-09-15,99.98174,1000
1,12347,2019-03-24,9249.73380,1001
2,12347,2019-11-01,54.58280,1002
3,12347,2019-11-02,2382.68120,1003
4,12348,2019-06-22,641.30382,1004
...,...,...,...,...
3203,18269,2019-04-05,98.52698,4203
3204,18269,2019-06-20,18.13394,4204
3205,18277,2019-10-23,301.02000,4205
3206,18283,2019-07-29,2090.81911,4206


In [315]:
df3=df2[['CustomerID','Transaction_Date','Basket_ID']].merge(trans_level_df,how='inner',on=['CustomerID','Transaction_Date'])
clv_df=df3.copy()

In [321]:
clv_df.groupby(['CustomerID','Basket_ID']).agg(total_sales_basket=('Trans_Total','sum'),transaction_date=('Transaction_Date','max'))


total_sales_basket transaction_date
CustomerID Basket_ID                                     
12346      1000                 99.98174       2019-09-15
12347      1001               9249.73380       2019-03-24
           1002                 54.58280       2019-11-01
           1003               2382.68120       2019-11-02
12348      1004                641.30382       2019-06-22
...                                  ...              ...
18269      4203                 98.52698       2019-04-05
           4204                 18.13394       2019-06-20
18277      4205                301.02000       2019-10-23
18283      4206               2090.81911       2019-07-29
           4207               4483.12416       2019-10-10

[3208 rows x 2 columns]

In [340]:
customer_sales_1_df= clv_df.groupby(['CustomerID','Basket_ID']).agg(total_sales_basket=('Trans_Total','sum'),transaction_date=('Transaction_Date','max')).reset_index()\
 .groupby('CustomerID').agg(time_days=('transaction_date',lambda x: (x.max()- x.min()).days), # time_days= no of days b/w first and last transaction of cust with us.
        frequency=('Basket_ID','nunique'),                                                    # count of unique basket_id i.e no of days he/she made atleast one visit/purchase
        total_sales=('total_sales_basket','sum'),                                             # Total Rev generated per customer
       avg_sales_per_trans=('total_sales_basket','mean')).reset_index()                                 # Avg Rev of each customer

In [341]:
customer_sales_1_df

,CustomerID,time_days,frequency,total_sales,avg_sales_per_trans
0,12346,0,1,99.98174,99.981740
1,12347,223,3,11686.99780,3895.665933
2,12348,119,2,1385.84620,692.923100
3,12350,0,1,1138.22394,1138.223940
4,12356,0,1,1270.45086,1270.450860
...,...,...,...,...,...
1463,18259,0,1,611.67360,611.673600
1464,18260,105,3,2181.64633,727.215443
1465,18269,76,2,116.66092,58.330460
1466,18277,0,1,301.02000,301.020000


In [366]:
summary_1={
    'average_order_value': customer_sales_1_df['avg_sales_per_trans'].mean(),  # Avg of "avg_sales_per_trans" for each customer
    'average_purchase_frequency': customer_sales_1_df['frequency'].mean(),    # This is avg
    'churn_rate': 1-(customer_sales_1_df['frequency']>1).sum()/len(customer_sales_1_df['frequency']),
    'max_days_bw_first_last_trans': customer_sales_1_df['time_days'].max()
}
summary_1_df=pd.DataFrame([summary_1])
summary_1_df

,average_order_value,average_purchase_frequency,churn_rate,max_days_bw_first_last_trans
0,1377.815327,2.185286,0.5,358


1. The churn rate calculated is based on frequency ie if a customer only made 
1 visits/trips throughout the time period of our data (1 jan 2019 till 31 dec 2019) will be called as churner.
2. Avg purchase frequency is total no of visits(or purchasesor transactions) made by customer/total no of customers.
It tells us the avg no of times a customer makes a purchase throughout a period (here 1 year).
3. max_days, this metric tells us the no of days between the first and last transaction for our customers. Its lower value may indicate customer's satisfaction with our business is good. 

In [371]:
# calculating clv now
customer_lifetime= 2 # This is defined as per historical data. I have assumed expected time for which customer will stay with us is 2 years.   


summary_1_df['clv_without_churn']= (summary_1_df['average_purchase_frequency']* summary_1_df['average_order_value'])/(summary_1_df['max_days_bw_first_last_trans']/365)\
                            * (customer_lifetime)

In [372]:
summary_1_df['clv_with_churn']= (summary_1_df['average_purchase_frequency']* summary_1_df['average_order_value'])/(summary_1_df['churn_rate'])


In [373]:
summary_1_df

,average_order_value,average_purchase_frequency,churn_rate,max_days_bw_first_last_trans,clv_without_churn,clv_with_churn
0,1377.815327,2.185286,0.5,358,6139.586879,6021.841377


# 5. Churn Rate

Calculation of churn varies form business to business.

- We can design a condition like " if a customer was inactive in past three months he should be treated as churner".
- we may also approach using transaction frequency, 
we calculate the avg days bw the transactions done by each customer (say a) then compare it with their days between the last transaction and current day (say b).
Based on this we can define the churned customer, if b> constant*a  then we may consider that cust as churned customer.  

In [12]:
df2

,CustomerID,Transaction_Date,total_spend_each_day,Basket_ID
0,12346,2019-09-15,99.98174,1000
1,12347,2019-03-24,9249.73380,1001
2,12347,2019-11-01,54.58280,1002
3,12347,2019-11-02,2382.68120,1003
4,12348,2019-06-22,641.30382,1004
...,...,...,...,...
3203,18269,2019-04-05,98.52698,4203
3204,18269,2019-06-20,18.13394,4204
3205,18277,2019-10-23,301.02000,4205
3206,18283,2019-07-29,2090.81911,4206


In [63]:
dff=df2.copy()
dff['prev_tran_date']=dff.groupby('CustomerID')['Transaction_Date'].shift(1)

dff['days_between']=(dff['Transaction_Date']-dff['prev_tran_date']).dt.days
# dff.days_between.fillna(dff.days_between.mean(),inplace=True)
dff

,CustomerID,Transaction_Date,total_spend_each_day,Basket_ID,prev_tran_date,days_between
0,12346,2019-09-15,99.98174,1000,NaT,NaN
1,12347,2019-03-24,9249.73380,1001,NaT,NaN
2,12347,2019-11-01,54.58280,1002,2019-03-24,222.0
3,12347,2019-11-02,2382.68120,1003,2019-11-01,1.0
4,12348,2019-06-22,641.30382,1004,NaT,NaN
...,...,...,...,...,...,...
3203,18269,2019-04-05,98.52698,4203,NaT,NaN
3204,18269,2019-06-20,18.13394,4204,2019-04-05,76.0
3205,18277,2019-10-23,301.02000,4205,NaT,NaN
3206,18283,2019-07-29,2090.81911,4206,NaT,NaN


In [66]:

cust_frequency=dff.groupby('CustomerID').days_between.mean().reset_index()
print(cust_frequency.days_between.mean() )

cust_frequency.rename(columns={'days_between':'Avg_days_bw'},inplace=True)
cust_frequency['Avg_days_bw'].fillna(cust_frequency['Avg_days_bw'].mean(),inplace=True)

64.92568308915416


C:\Users\welcome\AppData\Local\Temp\ipykernel_24520\2246824441.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cust_frequency['Avg_days_bw'].fillna(cust_frequency['Avg_days_bw'].mean(),inplace=True)


In [67]:
cust_frequency

,CustomerID,Avg_days_bw
0,12346,64.925683
1,12347,111.500000
2,12348,119.000000
3,12350,64.925683
4,12356,64.925683
...,...,...
1463,18259,64.925683
1464,18260,52.500000
1465,18269,76.000000
1466,18277,64.925683


In [68]:
dff = dff.merge(cust_frequency, on='CustomerID', how='left')

In [69]:


# Define a threshold (e.g., 2.5 times the average transaction frequency)
threshold_multiplier = 2.5
current_date = pd.to_datetime('2020-01-01')  # Bcz he data is from 1 jan 2019 to 31 dec 2019. Although it doesnt matter we can use actual
                                             # current date as well.

# Identify churned customers based on transaction frequency
dff['Last_Transaction_Date'] = dff.groupby('CustomerID')['Transaction_Date'].transform('max')
dff['Days_Since_Last_Transaction'] = (current_date - dff['Last_Transaction_Date']).dt.days
dff


,CustomerID,Transaction_Date,total_spend_each_day,Basket_ID,prev_tran_date,days_between,Avg_days_bw,Last_Transaction_Date,Days_Since_Last_Transaction
0,12346,2019-09-15,99.98174,1000,NaT,NaN,64.925683,2019-09-15,108
1,12347,2019-03-24,9249.73380,1001,NaT,NaN,111.500000,2019-11-02,60
2,12347,2019-11-01,54.58280,1002,2019-03-24,222.0,111.500000,2019-11-02,60
3,12347,2019-11-02,2382.68120,1003,2019-11-01,1.0,111.500000,2019-11-02,60
4,12348,2019-06-22,641.30382,1004,NaT,NaN,119.000000,2019-10-19,74
...,...,...,...,...,...,...,...,...,...
3203,18269,2019-04-05,98.52698,4203,NaT,NaN,76.000000,2019-06-20,195
3204,18269,2019-06-20,18.13394,4204,2019-04-05,76.0,76.000000,2019-06-20,195
3205,18277,2019-10-23,301.02000,4205,NaT,NaN,64.925683,2019-10-23,70
3206,18283,2019-07-29,2090.81911,4206,NaT,NaN,73.000000,2019-10-10,83


In [70]:
churned_customers = dff[dff['Days_Since_Last_Transaction'] > dff['Avg_days_bw'] * threshold_multiplier]['CustomerID'].unique()
churned_customers.size

698

In [71]:
churned_customers = dff[dff['Days_Since_Last_Transaction'] > dff['Avg_days_bw'] * threshold_multiplier]['CustomerID'].unique()

# Calculate churn rate
total_customers = dff['CustomerID'].nunique()
churn_rate = len(churned_customers) / total_customers

print(f'Churn rate: {np.round(churn_rate*100,2)}%')



Churn rate: 47.55%


# 6. Repeat Purchase Rate (RPR)

In [401]:
rpr=trans_level_df.groupby('CustomerID').agg(No_of_Trans=('Transaction_ID','size')).reset_index()

(rpr.No_of_Trans>1).sum()/len(rpr.No_of_Trans)

0.9148501362397821

In [410]:
dfs={
    'orders' : trans_level_df,
    'customers': pd.read_excel('CustomersData.xlsx'),
    'MER':mer,
    'CAC': cac,
    'CLV & AOV': customer_sales_1_df,
    'churn_rate': dff,
    'RPR': rpr
}

with pd.ExcelWriter('Summary.xlsx') as writer:
    for sheet_name, df in dfs.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

